# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Program: _Computer Systems Engineering_  </center>
---
### <center> **Spring 2025** </center>
---

**Lab 06**: Big Data Pipeline for Netflix data.

**Date**: 10/03/2025

**Team Name**: Par De Dos

**Professor**: Pablo Camarillo Ramirez

In [16]:
# Initialize findspark to get acces to de PySpark installation
import findspark
findspark.init()

In [17]:
from pyspark.sql import SparkSession

# Create connection to the spark cluster
spark = SparkSession.builder \
    .appName("Lab 06 - Par de Dos") \
    .master("spark://4231cbc2c134:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

# Create SparkContext
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [18]:
from team_ParDeDos.spark_utils import SparkUtils
netflix_schema = SparkUtils.generate_schema([
	("show_id", "string"),
	("type", "string"),
	("title", "string"),
	("director", "string"),
	("country", "string"),
	("date_added", "string"),
	("release_year", "integer"),
	("rating", "string"),
	("duration", "string"),
	("listed_in", "string")
])

netflix_df = spark.read.schema(netflix_schema).option("header", "true").csv("/home/jovyan/notebooks/data/netflix1.csv")

In [19]:
from pyspark.sql.functions import when, count, col
netflix_df.printSchema()
netflix_df.show(5, truncate=False)
netflix_df.select([count(when(col(c).isNull(), c)).alias(c) for c in netflix_df.columns]).show()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)



+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead            |Kirsten Johnson|United States|9/25/2021 |2020        |PG-13 |90 min  |Documentaries                                                |
|s3     |TV Show|Ganglands                       |Julien Leclercq|France       |9/24/2021 |2021        |TV-MA |1 Season|Crime TV Shows, International TV Shows, TV Action & Adventure|
|s6     |TV Show|Midnight Mass                   |Mike Flanagan  |United States|9/24/

+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|show_id|type|title|director|country|date_added|release_year|rating|duration|listed_in|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|      0|   0|    0|       1|      1|         1|           2|     1|       2|        2|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+



In [20]:
netflix_cleaned = SparkUtils.clean_df(netflix_df, netflix_schema)

netflix_cleaned.printSchema()
netflix_cleaned.show(5, truncate=False)
netflix_cleaned.select([count(when(col(c).isNull(), c)).alias(c) for c in netflix_cleaned.columns]).show()


root
 |-- show_id: string (nullable = false)
 |-- type: string (nullable = false)
 |-- title: string (nullable = false)
 |-- director: string (nullable = false)
 |-- country: string (nullable = false)
 |-- date_added: string (nullable = false)
 |-- release_year: integer (nullable = false)
 |-- rating: string (nullable = false)
 |-- duration: string (nullable = false)
 |-- listed_in: string (nullable = false)

+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead 

In [21]:

params = {
	"mode": "overwrite",
	"dataframe": netflix_cleaned,
	"path": "/home/jovyan/notebooks/data/lab_06_ParDeDos/",
	"criteria": ["release_year"]
}

SparkUtils.write_df(params)

In [23]:
!ls notebooks/data/lab_06_ParDeDos/ | wc -l

76


In [15]:
# Stop the SparkContext
sc.stop()